# ETAPA 1 - Análise Descritiva da Exposição de IA Generativa com ILO Index e PNADc
## PREPARAÇÃO DOS DADOS 


**Dissertação:** Inteligência Artificial Generativa e o Mercado de Trabalho Brasileiro: Uma Análise de Exposição Ocupacional e seus Efeitos Distributivos.
**Aluno:** Manoel Brasil Orlandi

### Obejtivo

Construir a base analítica que une PNAD Contínua e o índice de exposição à IA (ILO), com ocupações em COD e ISCO-08.

**Entradas:** Microdados PNAD (BigQuery), planilha ILO, estrutura COD.  
**Saída principal:** `data/output/pnad_ilo_merged.csv`

### 1. Configuração do ambiente
Definir caminhos, importar bibliotecas e configurar logs. 

In [1]:
# Instalar dependências no kernel atual (executar apenas uma vez)
%pip install pandas numpy pyarrow openpyxl --quiet


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Etapa 1.1 - Preparação de Dados - Configuração do ambiente

import warnings
import pandas as pd
import numpy as np
from pathlib import Path

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------------------------------------
# Caminhos (relativos ao diretório do notebook)
# ---------------------------------------------------------------------------
DATA_INPUT     = Path("data/input")
DATA_RAW       = Path("data/raw")
DATA_PROCESSED = Path("data/processed")
DATA_OUTPUT    = Path("data/output")

for d in [DATA_RAW, DATA_PROCESSED, DATA_OUTPUT]:
    d.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------------
# Parâmetros PNAD / GCP
# ---------------------------------------------------------------------------
GCP_PROJECT_ID  = "mestrado-pnad-2026"
PNAD_ANO        = 2025
PNAD_TRIMESTRE  = 3

# ---------------------------------------------------------------------------
# Arquivo ILO (já copiado para data/input)
# ---------------------------------------------------------------------------
ILO_FILE = DATA_INPUT / "Final_Scores_ISCO08_Gmyrek_et_al_2025.xlsx"

# ---------------------------------------------------------------------------
# Mapeamentos
# ---------------------------------------------------------------------------
REGIAO_MAP = {
    'RO': 'Norte', 'AC': 'Norte', 'AM': 'Norte', 'RR': 'Norte',
    'PA': 'Norte', 'AP': 'Norte', 'TO': 'Norte',
    'MA': 'Nordeste', 'PI': 'Nordeste', 'CE': 'Nordeste', 'RN': 'Nordeste',
    'PB': 'Nordeste', 'PE': 'Nordeste', 'AL': 'Nordeste', 'SE': 'Nordeste', 'BA': 'Nordeste',
    'MG': 'Sudeste', 'ES': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'SC': 'Sul', 'RS': 'Sul',
    'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'DF': 'Centro-Oeste',
}

GRANDES_GRUPOS = {
    '1': 'Dirigentes e gerentes',
    '2': 'Profissionais das ciências',
    '3': 'Técnicos nível médio',
    '4': 'Apoio administrativo',
    '5': 'Serviços e vendedores',
    '6': 'Agropecuária qualificada',
    '7': 'Indústria qualificada',
    '8': 'Operadores de máquinas',
    '9': 'Ocupações elementares',
}

RACA_AGREGADA_MAP = {
    '1': 'Branca',
    '2': 'Negra',   # Preta
    '4': 'Negra',   # Parda
    '3': 'Outras',  # Amarela
    '5': 'Outras',  # Indígena
    '9': 'Outras',  # Sem declaração
}

POSICAO_FORMAL = ['1', '3', '5']  # Empregado c/ carteira, Militar, Empregador

IDADE_BINS   = [0, 25, 35, 45, 55, 100]
IDADE_LABELS = ['18-24', '25-34', '35-44', '45-54', '55+']

# ---------------------------------------------------------------------------
# Funções utilitárias – estatísticas ponderadas
# ---------------------------------------------------------------------------
def weighted_mean(values, weights):
    """Média ponderada (ignora NaN)."""
    mask = ~(pd.isna(values) | pd.isna(weights))
    if mask.sum() == 0:
        return np.nan
    return np.average(values[mask], weights=weights[mask])

def weighted_std(values, weights):
    """Desvio-padrão ponderado (ignora NaN)."""
    mask = ~(pd.isna(values) | pd.isna(weights))
    if mask.sum() == 0:
        return np.nan
    avg = np.average(values[mask], weights=weights[mask])
    variance = np.average((values[mask] - avg) ** 2, weights=weights[mask])
    return np.sqrt(variance)

print("Configuração carregada com sucesso.")
print(f"  PNAD: {PNAD_ANO} Q{PNAD_TRIMESTRE}")
print(f"  Projeto GCP: {GCP_PROJECT_ID}")
print(f"  ILO file: {ILO_FILE} (existe: {ILO_FILE.exists()})")

Configuração carregada com sucesso.
  PNAD: 2025 Q3
  Projeto GCP: mestrado-pnad-2026
  ILO file: data/input/Final_Scores_ISCO08_Gmyrek_et_al_2025.xlsx (existe: True)


### 2a. Download dos microdados PNAD
Extrair da PNAD Contínua (BigQuery) as variáveis necessárias para o trimestre/ano definido.
**Saída:** `data/raw/pnad_*.parquet`

In [3]:
# Etapa 1.2a - Preparação de Dados - Download dos microdados PNAD
# Lógica: se o parquet já existe em data/raw/, carrega direto; senão, baixa do BigQuery.

pnad_files = sorted(DATA_RAW.glob("pnad_*.parquet"))

if pnad_files:
    # --- Caminho rápido: arquivo local já disponível ---
    pnad_path = pnad_files[-1]  # mais recente
    print(f"Arquivo PNAD encontrado localmente: {pnad_path.name}")
    df_pnad_raw = pd.read_parquet(pnad_path)
    print(f"Carregado: {len(df_pnad_raw):,} observações")

else:
    # --- Caminho completo: download via BigQuery ---
    print("Nenhum arquivo PNAD local encontrado. Iniciando download do BigQuery...")
    import basedosdados as bd

    # Verificar trimestres disponíveis
    query_check = """
    SELECT DISTINCT ano, trimestre, COUNT(*) as n_obs
    FROM `basedosdados.br_ibge_pnadc.microdados`
    WHERE ano >= 2024
    GROUP BY ano, trimestre
    ORDER BY ano DESC, trimestre DESC
    LIMIT 5
    """
    df_check = bd.read_sql(query_check, billing_project_id=GCP_PROJECT_ID)
    print(f"Trimestres disponíveis:\n{df_check}")

    trimestre_existe = len(
        df_check[(df_check['ano'] == PNAD_ANO) & (df_check['trimestre'] == PNAD_TRIMESTRE)]
    ) > 0

    if trimestre_existe:
        ano_usar, trim_usar = PNAD_ANO, PNAD_TRIMESTRE
    else:
        ano_usar = int(df_check.iloc[0]['ano'])
        trim_usar = int(df_check.iloc[0]['trimestre'])
        print(f"AVISO: {PNAD_ANO} Q{PNAD_TRIMESTRE} indisponível. Usando {ano_usar} Q{trim_usar}")

    # Query principal
    query = f"""
    SELECT
        ano,
        trimestre,
        sigla_uf,
        v2007  AS sexo,
        v2009  AS idade,
        v2010  AS raca_cor,
        vd3004 AS nivel_instrucao,
        v4010  AS cod_ocupacao,
        v4013  AS grupamento_atividade,
        vd4009 AS posicao_ocupacao,
        vd4020 AS rendimento_habitual,
        vd4016 AS rendimento_todos,
        v4019  AS horas_trabalhadas,
        v1028  AS peso
    FROM `basedosdados.br_ibge_pnadc.microdados`
    WHERE ano = {ano_usar}
      AND trimestre = {trim_usar}
      AND v4010 IS NOT NULL
      AND vd4020 > 0
    """

    print(f"Executando query para {ano_usar} Q{trim_usar} (pode demorar 2-5 min)...")
    df_pnad_raw = bd.read_sql(query, billing_project_id=GCP_PROJECT_ID)

    # Salvar parquet
    ano_real = int(df_pnad_raw['ano'].iloc[0])
    trim_real = int(df_pnad_raw['trimestre'].iloc[0])
    output_path = DATA_RAW / f"pnad_{ano_real}q{trim_real}.parquet"
    df_pnad_raw.to_parquet(output_path, index=False)
    print(f"Salvo em: {output_path}")

print(f"\ndf_pnad_raw: {df_pnad_raw.shape[0]:,} linhas x {df_pnad_raw.shape[1]} colunas")

Arquivo PNAD encontrado localmente: pnad_2025q2.parquet
Carregado: 202,339 observações

df_pnad_raw: 202,339 linhas x 14 colunas


### 2b. Verificar dados microdados PNAD (CHECKPOINT)
Verificar dados gerados

In [4]:
# Etapa 1.2b - Preparação de Dados - Verificar dados microdados PNAD

print("=" * 60)
print("CHECKPOINT - Microdados PNAD")
print("=" * 60)

print(f"\nShape: {df_pnad_raw.shape}")
print(f"Colunas: {list(df_pnad_raw.columns)}")

# UFs
n_ufs = df_pnad_raw['sigla_uf'].nunique()
print(f"\nUFs presentes: {n_ufs}")
if n_ufs != 27:
    print(f"  WARNING: Esperado 27 UFs, encontrado {n_ufs}")

# População
pop_milhoes = df_pnad_raw['peso'].sum() / 1e6
print(f"População representada: {pop_milhoes:.1f} milhões")

# Linhas
if len(df_pnad_raw) < 100_000:
    print(f"  WARNING: Apenas {len(df_pnad_raw):,} linhas (esperado > 100.000)")

# Tipos
print(f"\nDtypes:\n{df_pnad_raw.dtypes}")

# Amostra
print("\nPrimeiras linhas:")
df_pnad_raw.head()

CHECKPOINT - Microdados PNAD

Shape: (202339, 14)
Colunas: ['ano', 'trimestre', 'sigla_uf', 'sexo', 'idade', 'raca_cor', 'nivel_instrucao', 'cod_ocupacao', 'grupamento_atividade', 'posicao_ocupacao', 'rendimento_habitual', 'rendimento_todos', 'horas_trabalhadas', 'peso']

UFs presentes: 27
População representada: 99.0 milhões

Dtypes:
ano                       Int64
trimestre                 Int64
sigla_uf                 object
sexo                     object
idade                     Int64
raca_cor                 object
nivel_instrucao          object
cod_ocupacao             object
grupamento_atividade     object
posicao_ocupacao         object
rendimento_habitual     float64
rendimento_todos        float64
horas_trabalhadas        object
peso                    float64
dtype: object

Primeiras linhas:


,ano,trimestre,sigla_uf,sexo,idade,raca_cor,nivel_instrucao,cod_ocupacao,grupamento_atividade,posicao_ocupacao,rendimento_habitual,rendimento_todos,horas_trabalhadas,peso
0,2025,2,SP,1,18,1,4,5223,48030,1,1650.0,1650.0,None,819.403494
1,2025,2,SC,1,17,4,4,4321,29002,1,2300.0,2300.0,None,247.272091
2,2025,2,MS,1,18,2,5,7231,45020,1,1900.0,1900.0,None,544.629439
3,2025,2,DF,1,19,1,5,5223,48071,1,1518.0,1518.0,None,392.977263
4,2025,2,RS,1,18,4,5,8211,15020,1,2000.0,2000.0,None,570.385886


### 3a. Processar índice de exposição ILO
Lê a planilha ILO com scores de exposição por ISCO-08, padroniza e gera níveis

In [5]:
# Etapa 1.3a - Preparação de Dados - Processar índice de exposição ILO

print(f"Lendo arquivo ILO: {ILO_FILE}")
df_ilo_raw = pd.read_excel(ILO_FILE)
print(f"Linhas raw (tarefas): {len(df_ilo_raw):,}")
print(f"Colunas disponíveis: {list(df_ilo_raw.columns)}")

# Mapeamento de colunas
col_mapping = {
    'ISCO_08': 'isco_08',
    'Title': 'occupation_title',
    'mean_score_2025': 'exposure_score',
    'SD_2025': 'exposure_sd',
    'potential25': 'exposure_gradient',
}

available_cols = [c for c in col_mapping.keys() if c in df_ilo_raw.columns]
print(f"Colunas mapeadas: {available_cols}")

df_ilo_renamed = df_ilo_raw.rename(
    columns={k: v for k, v in col_mapping.items() if k in df_ilo_raw.columns}
)

# Agregar por ocupação (arquivo original tem múltiplas tarefas por ocupação)
df_ilo = df_ilo_renamed.groupby('isco_08').agg({
    'occupation_title': 'first',
    'exposure_score': 'mean',
    'exposure_sd': 'mean',
    'exposure_gradient': 'first',
}).reset_index()

# Garantir formato string com 4 dígitos
df_ilo['isco_08_str'] = df_ilo['isco_08'].astype(str).str.zfill(4)

print(f"\nOcupações únicas: {len(df_ilo):,}")
print(f"Score médio: {df_ilo['exposure_score'].mean():.3f}")
print(f"Score range: [{df_ilo['exposure_score'].min():.3f}, {df_ilo['exposure_score'].max():.3f}]")

# Salvar processado
ilo_output = DATA_PROCESSED / "ilo_exposure_clean.csv"
df_ilo.to_csv(ilo_output, index=False)
print(f"\nSalvo em: {ilo_output}")

Lendo arquivo ILO: data/input/Final_Scores_ISCO08_Gmyrek_et_al_2025.xlsx
Linhas raw (tarefas): 3,265
Colunas disponíveis: ['label4d', 'label1d', 'ISCO_08', 'Title', 'taskID', 'Task_ISCO', 'score_2023', 'Weaviate Status', 'predicted_score_2025_gpt4o', 'prediction_justification_gpt4o', 'weaviate_status_gemini', 'predicted_score_2025_gemini', 'prediction_justification_gemini', 'score_2025', 'source', 'mean_score_2023', 'mean_score_2025', 'SD_2023', 'SD_2025', 'potential25', 'potential23']
Colunas mapeadas: ['ISCO_08', 'Title', 'mean_score_2025', 'SD_2025', 'potential25']

Ocupações únicas: 427
Score médio: 0.297
Score range: [0.090, 0.700]

Salvo em: data/processed/ilo_exposure_clean.csv


### 3b. Verificar índice de exposição ILO
Verificar: número de ocupações, coluna de score, distribuição por gradiente

In [6]:
# Etapa 1.3b - Preparação de Dados - Verificar índice de exposição ILO

print("=" * 60)
print("CHECKPOINT - Índice ILO")
print("=" * 60)

# Número de ocupações
n_ocup = len(df_ilo)
print(f"\nOcupações: {n_ocup}")
if n_ocup < 400:
    print(f"  WARNING: Poucas ocupações ({n_ocup}). Esperado ~427.")

# Range de scores
score_min = df_ilo['exposure_score'].min()
score_max = df_ilo['exposure_score'].max()
print(f"Score range: [{score_min:.3f}, {score_max:.3f}]")
if score_min < 0 or score_max > 1:
    print(f"  WARNING: Scores fora do intervalo [0, 1]")

# Distribuição por gradiente
print("\nDistribuição por gradiente:")
for grad, count in df_ilo['exposure_gradient'].value_counts().items():
    print(f"  {grad}: {count} ocupações")

# Amostra
print("\nAmostra (5 maiores scores):")
df_ilo.nlargest(5, 'exposure_score')[['isco_08_str', 'occupation_title', 'exposure_score']]

CHECKPOINT - Índice ILO

Ocupações: 427
Score range: [0.090, 0.700]

Distribuição por gradiente:
  Not Exposed: 231 ocupações
  Minimal Exposure: 84 ocupações
  Exposed: Gradient 2: 44 ocupações
  Exposed: Gradient 3: 38 ocupações
  Exposed: Gradient 1: 17 ocupações
  Exposed: Gradient 4: 13 ocupações

Amostra (5 maiores scores):


,isco_08_str,occupation_title,exposure_score
207,4132,Data Entry Clerks,0.70
206,4131,Typists and Word Processing Operators,0.65
220,4311,Accounting and Bookkeeping Clerks,0.64
221,4312,"Statistical, Finance and Insurance Clerks",0.64
164,3311,Securities and Finance Dealers and Brokers,0.63


### 4a. Limpeza e variáveis derivadas – PNAD

Filtra população de interesse, cria variáveis derivadas (região, grandes grupos COD, quintis de renda, etc.) e padroniza códigos de ocupação.

**Entrada:** `data/raw/pnad_*.parquet`.  
**Saída:** `data/processed/pnad_clean.csv`

In [7]:
# Etapa 1.4a - Preparação de Dados - Limpeza e variáveis derivadas

df_pnad = df_pnad_raw.copy()
n_inicial = len(df_pnad)
print(f"Observações iniciais: {n_inicial:,}")

# ---------------------------------------------------------------------------
# LIMPEZA
# ---------------------------------------------------------------------------

# 1. Converter tipos
df_pnad['cod_ocupacao'] = df_pnad['cod_ocupacao'].astype(str).str.zfill(4)
df_pnad['idade'] = pd.to_numeric(df_pnad['idade'], errors='coerce')
df_pnad['rendimento_habitual'] = pd.to_numeric(df_pnad['rendimento_habitual'], errors='coerce')
df_pnad['peso'] = pd.to_numeric(df_pnad['peso'], errors='coerce')

# 2. Remover missings críticos
df_pnad = df_pnad.dropna(subset=['cod_ocupacao', 'idade', 'peso'])
print(f"Após remover missings críticos: {len(df_pnad):,} ({len(df_pnad)/n_inicial:.1%})")

# 3. Filtrar faixa etária (18-65)
df_pnad = df_pnad[(df_pnad['idade'] >= 18) & (df_pnad['idade'] <= 65)]
print(f"Após filtrar 18-65 anos: {len(df_pnad):,} ({len(df_pnad)/n_inicial:.1%})")

# 4. Remover ocupações inválidas
df_pnad = df_pnad[~df_pnad['cod_ocupacao'].isin(['0000', '9999'])]
print(f"Após remover ocupações inválidas: {len(df_pnad):,}")

# ---------------------------------------------------------------------------
# VARIÁVEIS DERIVADAS
# ---------------------------------------------------------------------------

# Formalidade
df_pnad['formal'] = df_pnad['posicao_ocupacao'].astype(str).isin(POSICAO_FORMAL).astype(int)
print(f"\nTaxa de formalidade: {df_pnad['formal'].mean():.1%}")

# Faixas etárias
df_pnad['faixa_etaria'] = pd.cut(
    df_pnad['idade'], bins=IDADE_BINS, labels=IDADE_LABELS
)

# Região
df_pnad['regiao'] = df_pnad['sigla_uf'].map(REGIAO_MAP)

# Raça agregada
df_pnad['raca_agregada'] = df_pnad['raca_cor'].astype(str).map(RACA_AGREGADA_MAP)

# Grande grupo ocupacional
df_pnad['grande_grupo'] = df_pnad['cod_ocupacao'].str[0].map(GRANDES_GRUPOS)

# Sexo como texto
df_pnad['sexo_texto'] = df_pnad['sexo'].map({1: 'Homem', 2: 'Mulher', '1': 'Homem', '2': 'Mulher'})

# Winsorização de renda (percentil 99)
p99 = df_pnad['rendimento_habitual'].quantile(0.99)
df_pnad['rendimento_winsor'] = df_pnad['rendimento_habitual'].clip(upper=p99)
print(f"Percentil 99 renda: R$ {p99:,.0f}")

# ---------------------------------------------------------------------------
# SALVAR
# ---------------------------------------------------------------------------
pnad_clean_path = DATA_PROCESSED / "pnad_clean.csv"
df_pnad.to_csv(pnad_clean_path, index=False)
print(f"\nSalvo em: {pnad_clean_path}")
print(f"df_pnad: {df_pnad.shape[0]:,} linhas x {df_pnad.shape[1]} colunas")

Observações iniciais: 202,339
Após remover missings críticos: 202,339 (100.0%)
Após filtrar 18-65 anos: 192,245 (95.0%)
Após remover ocupações inválidas: 192,235

Taxa de formalidade: 38.2%
Percentil 99 renda: R$ 21,500

Salvo em: data/processed/pnad_clean.csv
df_pnad: 192,235 linhas x 21 colunas


### 4b. Verificar Limpeza e variáveis derivadas – PNAD
Verificar: número de linhas, colunas criadas, valores faltantes em COD.

In [8]:
# Etapa 1.4b - Preparação de Dados - Verificar Limpeza e variáveis derivadas

print("=" * 60)
print("CHECKPOINT - Limpeza PNAD")
print("=" * 60)

# Perda de observações
pct_perda = 1 - len(df_pnad) / n_inicial
print(f"\nObservações: {n_inicial:,} -> {len(df_pnad):,} (perda: {pct_perda:.1%})")
if pct_perda > 0.20:
    print(f"  WARNING: Perda de {pct_perda:.1%} das observações (> 20%)")

# Missings em variáveis derivadas
for col in ['regiao', 'raca_agregada', 'grande_grupo', 'faixa_etaria', 'sexo_texto']:
    n_miss = df_pnad[col].isna().sum()
    if n_miss > 0:
        print(f"  WARNING: {col} tem {n_miss:,} valores faltantes")

# Distribuição por sexo
print(f"\nOcupações únicas (COD): {df_pnad['cod_ocupacao'].nunique()}")
print(f"UFs: {df_pnad['sigla_uf'].nunique()}")
print(f"População representada: {df_pnad['peso'].sum()/1e6:.1f} milhões")

print("\nDistribuição por sexo:")
for sexo, peso in df_pnad.groupby('sexo_texto')['peso'].sum().items():
    print(f"  {sexo}: {peso/1e6:.1f} milhões")

print("\nDistribuição por região:")
for regiao, peso in df_pnad.groupby('regiao')['peso'].sum().sort_values(ascending=False).items():
    print(f"  {regiao}: {peso/1e6:.1f} milhões")

print("\nDistribuição por faixa etária:")
print(df_pnad['faixa_etaria'].value_counts().sort_index())

CHECKPOINT - Limpeza PNAD

Observações: 202,339 -> 192,235 (perda: 5.0%)

Ocupações únicas (COD): 431
UFs: 27
População representada: 94.9 milhões

Distribuição por sexo:
  Homem: 53.4 milhões
  Mulher: 41.5 milhões

Distribuição por região:
  Sudeste: 42.9 milhões
  Nordeste: 21.3 milhões
  Sul: 15.2 milhões
  Centro-Oeste: 8.3 milhões
  Norte: 7.2 milhões

Distribuição por faixa etária:
faixa_etaria
18-24    27371
25-34    45008
35-44    51730
45-54    42650
55+      25476
Name: count, dtype: int64


### 5a. Crosswalk COD → ISCO-08
Mapear códigos de ocupação COD (PNAD) para ISCO-08 para permitir o merge com o índice ILO. Pode ser hierárquico (4 → 3 → 2 → 1 dígito) conforme implementado no script.


In [9]:
# Etapa 1.5a - Preparação de Dados - Crosswalk COD → ISCO-08

# Garantir formatos string
df_ilo['isco_08_str'] = df_ilo['isco_08_str'].astype(str).str.zfill(4)
df_pnad['cod_ocupacao'] = df_pnad['cod_ocupacao'].astype(str).str.zfill(4)

print(f"PNAD: {len(df_pnad):,} observações")
print(f"ILO:  {len(df_ilo):,} ocupações ISCO-08")

# ---------------------------------------------------------------------------
# Criar dicionários de lookup em cada nível hierárquico
# ---------------------------------------------------------------------------
ilo_4d = df_ilo.groupby('isco_08_str')['exposure_score'].mean().to_dict()
ilo_3d = df_ilo.groupby(df_ilo['isco_08_str'].str[:3])['exposure_score'].mean().to_dict()
ilo_2d = df_ilo.groupby(df_ilo['isco_08_str'].str[:2])['exposure_score'].mean().to_dict()
ilo_1d = df_ilo.groupby(df_ilo['isco_08_str'].str[:1])['exposure_score'].mean().to_dict()

print(f"\nCódigos ILO: 4d={len(ilo_4d)}, 3d={len(ilo_3d)}, 2d={len(ilo_2d)}, 1d={len(ilo_1d)}")

# ---------------------------------------------------------------------------
# Crosswalk hierárquico (4 → 3 → 2 → 1 dígito)
# ---------------------------------------------------------------------------
df_crosswalked = df_pnad.copy()
df_crosswalked['exposure_score'] = np.nan
df_crosswalked['match_level'] = None

# Nível 4-digit
mask_4d = df_crosswalked['cod_ocupacao'].isin(ilo_4d.keys())
df_crosswalked.loc[mask_4d, 'exposure_score'] = df_crosswalked.loc[mask_4d, 'cod_ocupacao'].map(ilo_4d)
df_crosswalked.loc[mask_4d, 'match_level'] = '4-digit'
print(f"\nMatch 4-digit: {mask_4d.sum():,} ({mask_4d.mean():.1%})")

# Nível 3-digit
mask_missing = df_crosswalked['exposure_score'].isna()
cod_3d = df_crosswalked.loc[mask_missing, 'cod_ocupacao'].str[:3]
mask_3d = cod_3d.isin(ilo_3d.keys())
idx_3d = mask_missing[mask_missing].index[mask_3d.values]
df_crosswalked.loc[idx_3d, 'exposure_score'] = cod_3d[mask_3d].map(ilo_3d).values
df_crosswalked.loc[idx_3d, 'match_level'] = '3-digit'
print(f"Match 3-digit: {len(idx_3d):,} ({len(idx_3d)/len(df_crosswalked):.1%})")

# Nível 2-digit
mask_missing = df_crosswalked['exposure_score'].isna()
cod_2d = df_crosswalked.loc[mask_missing, 'cod_ocupacao'].str[:2]
mask_2d = cod_2d.isin(ilo_2d.keys())
idx_2d = mask_missing[mask_missing].index[mask_2d.values]
df_crosswalked.loc[idx_2d, 'exposure_score'] = cod_2d[mask_2d].map(ilo_2d).values
df_crosswalked.loc[idx_2d, 'match_level'] = '2-digit'
print(f"Match 2-digit: {len(idx_2d):,} ({len(idx_2d)/len(df_crosswalked):.1%})")

# Nível 1-digit
mask_missing = df_crosswalked['exposure_score'].isna()
cod_1d = df_crosswalked.loc[mask_missing, 'cod_ocupacao'].str[:1]
mask_1d = cod_1d.isin(ilo_1d.keys())
idx_1d = mask_missing[mask_missing].index[mask_1d.values]
df_crosswalked.loc[idx_1d, 'exposure_score'] = cod_1d[mask_1d].map(ilo_1d).values
df_crosswalked.loc[idx_1d, 'match_level'] = '1-digit'
print(f"Match 1-digit: {len(idx_1d):,} ({len(idx_1d)/len(df_crosswalked):.1%})")

# Sem match
n_sem_match = df_crosswalked['exposure_score'].isna().sum()
print(f"Sem match:     {n_sem_match:,} ({n_sem_match/len(df_crosswalked):.1%})")

PNAD: 192,235 observações
ILO:  427 ocupações ISCO-08

Códigos ILO: 4d=427, 3d=127, 2d=40, 1d=9

Match 4-digit: 188,206 (97.9%)
Match 3-digit: 2,345 (1.2%)
Match 2-digit: 0 (0.0%)
Match 1-digit: 0 (0.0%)
Sem match:     1,684 (0.9%)


### 5a. Verificar Crosswalk COD → ISCO-08
Verificar: cobertura do crosswalk (percentual de linhas com ISCO preenchido).

In [10]:
# Etapa 1.5b - Preparação de Dados - Verificar Crosswalk COD → ISCO-08

print("=" * 60)
print("CHECKPOINT - Crosswalk COD → ISCO-08")
print("=" * 60)

# Cobertura total
coverage = df_crosswalked['exposure_score'].notna().mean()
print(f"\nCobertura total: {coverage:.1%}")
if coverage < 0.90:
    print(f"  WARNING: Cobertura {coverage:.1%} abaixo de 90%")

# Distribuição por nível de match
print("\nDistribuição por nível de match:")
for level, count in df_crosswalked['match_level'].value_counts().items():
    pct = count / len(df_crosswalked) * 100
    print(f"  {level}: {count:,} ({pct:.1f}%)")

# Estatísticas de score
print(f"\nEstatísticas do exposure_score:")
print(f"  Média:  {df_crosswalked['exposure_score'].mean():.3f}")
print(f"  Std:    {df_crosswalked['exposure_score'].std():.3f}")
print(f"  Min:    {df_crosswalked['exposure_score'].min():.3f}")
print(f"  Max:    {df_crosswalked['exposure_score'].max():.3f}")

# Sanity check: exposição por grande grupo
print("\nExposição média por grande grupo (sanity check):")
exp_grupos = df_crosswalked.groupby('grande_grupo').apply(
    lambda x: weighted_mean(x['exposure_score'].dropna(), x.loc[x['exposure_score'].notna(), 'peso'])
).sort_values(ascending=False)

for grupo, score in exp_grupos.items():
    print(f"  {grupo}: {score:.3f}")

# Validações de sanidade
print("\nVALIDAÇÃO DE SANIDADE:")
if 'Profissionais das ciências' in exp_grupos.index:
    val = exp_grupos['Profissionais das ciências']
    if val > 0.30:
        print(f"  OK - Profissionais das ciências com exposição ALTA ({val:.3f})")
    else:
        print(f"  WARNING: Profissionais das ciências com exposição BAIXA ({val:.3f}). Esperado > 0.30")

if 'Ocupações elementares' in exp_grupos.index:
    val = exp_grupos['Ocupações elementares']
    if val < 0.20:
        print(f"  OK - Ocupações elementares com exposição BAIXA ({val:.3f})")
    else:
        print(f"  WARNING: Ocupações elementares com exposição ALTA ({val:.3f}). Esperado < 0.20")

CHECKPOINT - Crosswalk COD → ISCO-08

Cobertura total: 99.1%

Distribuição por nível de match:
  4-digit: 188,206 (97.9%)
  3-digit: 2,345 (1.2%)

Estatísticas do exposure_score:
  Média:  0.267
  Std:    0.145
  Min:    0.090
  Max:    0.700

Exposição média por grande grupo (sanity check):
  Apoio administrativo: 0.553
  Dirigentes e gerentes: 0.400
  Profissionais das ciências: 0.352
  Técnicos nível médio: 0.345
  Serviços e vendedores: 0.306
  Operadores de máquinas: 0.223
  Agropecuária qualificada: 0.174
  Indústria qualificada: 0.152
  Ocupações elementares: 0.131

VALIDAÇÃO DE SANIDADE:
  OK - Profissionais das ciências com exposição ALTA (0.352)
  OK - Ocupações elementares com exposição BAIXA (0.131)


### 6. Merge final – PNAD + índice ILO
Juntar a base PNAD (com ISCO-08) ao índice ILO por código de ocupação. Gera a base analítica final da Etapa 1.
**Saída:** `data/output/pnad_ilo_merged.csv`

In [11]:
# Etapa 1.6 - Preparação de Dados - Merge final PNAD + ILO

df_final = df_crosswalked.copy()

# ---------------------------------------------------------------------------
# Classificação por gradiente ILO
# ---------------------------------------------------------------------------
def classify_gradient(score):
    if pd.isna(score):
        return 'Sem classificação'
    elif score < 0.22:
        return 'Not Exposed'
    elif score < 0.36:
        return 'Minimal Exposure'
    elif score < 0.45:
        return 'Gradient 1-2'
    elif score < 0.55:
        return 'Gradient 3'
    else:
        return 'Gradient 4 (Alta)'

df_final['exposure_gradient'] = df_final['exposure_score'].apply(classify_gradient)

print("Distribuição por gradiente:")
for grad, peso in df_final.groupby('exposure_gradient')['peso'].sum().sort_values(ascending=False).items():
    print(f"  {grad}: {peso/1e6:.1f} milhões")

# ---------------------------------------------------------------------------
# Quintis e decis de exposição
# ---------------------------------------------------------------------------
mask_valid = df_final['exposure_score'].notna()

df_final.loc[mask_valid, 'quintil_exposure'] = pd.qcut(
    df_final.loc[mask_valid, 'exposure_score'],
    q=5,
    labels=['Q1 (Baixa)', 'Q2', 'Q3', 'Q4', 'Q5 (Alta)'],
    duplicates='drop',
)

df_final.loc[mask_valid, 'decil_exposure'] = pd.qcut(
    df_final.loc[mask_valid, 'exposure_score'],
    q=10,
    labels=[f'D{i}' for i in range(1, 11)],
    duplicates='drop',
)

# ---------------------------------------------------------------------------
# Agregação setorial
# ---------------------------------------------------------------------------
setor_map = {
    '01': 'Agropecuária', '02': 'Agropecuária', '03': 'Agropecuária',
    '05': 'Indústria', '06': 'Indústria', '07': 'Indústria', '08': 'Indústria',
    '10': 'Indústria', '11': 'Indústria', '12': 'Indústria', '13': 'Indústria',
    '14': 'Indústria', '15': 'Indústria', '16': 'Indústria', '17': 'Indústria',
    '18': 'Indústria', '19': 'Indústria', '20': 'Indústria', '21': 'Indústria',
    '22': 'Indústria', '23': 'Indústria', '24': 'Indústria', '25': 'Indústria',
    '26': 'Indústria', '27': 'Indústria', '28': 'Indústria', '29': 'Indústria',
    '30': 'Indústria', '31': 'Indústria', '32': 'Indústria', '33': 'Indústria',
    '41': 'Construção', '42': 'Construção', '43': 'Construção',
    '45': 'Comércio', '46': 'Comércio', '47': 'Comércio',
    '49': 'Serviços', '50': 'Serviços', '51': 'Serviços', '52': 'Serviços',
    '53': 'Serviços', '55': 'Serviços', '56': 'Serviços',
    '58': 'TIC e Serviços Prof.', '59': 'TIC e Serviços Prof.', '60': 'TIC e Serviços Prof.',
    '61': 'TIC e Serviços Prof.', '62': 'TIC e Serviços Prof.', '63': 'TIC e Serviços Prof.',
    '64': 'TIC e Serviços Prof.', '65': 'TIC e Serviços Prof.', '66': 'TIC e Serviços Prof.',
    '69': 'TIC e Serviços Prof.', '70': 'TIC e Serviços Prof.', '71': 'TIC e Serviços Prof.',
    '72': 'TIC e Serviços Prof.', '73': 'TIC e Serviços Prof.', '74': 'TIC e Serviços Prof.',
    '75': 'TIC e Serviços Prof.',
    '84': 'Administração Pública',
    '85': 'Educação',
    '86': 'Saúde', '87': 'Saúde', '88': 'Saúde',
}

df_final['cnae_2d'] = df_final['grupamento_atividade'].astype(str).str[:2]
df_final['setor_agregado'] = df_final['cnae_2d'].map(setor_map).fillna('Outros Serviços')

# ---------------------------------------------------------------------------
# Selecionar colunas finais e salvar
# ---------------------------------------------------------------------------
cols_output = [
    'ano', 'trimestre', 'sigla_uf', 'regiao',
    'sexo', 'sexo_texto', 'idade', 'faixa_etaria',
    'raca_cor', 'raca_agregada', 'nivel_instrucao',
    'cod_ocupacao', 'grande_grupo',
    'grupamento_atividade', 'setor_agregado',
    'posicao_ocupacao', 'formal',
    'rendimento_habitual', 'rendimento_winsor', 'rendimento_todos',
    'horas_trabalhadas', 'peso',
    'exposure_score', 'exposure_gradient', 'match_level',
    'quintil_exposure', 'decil_exposure',
]

df_final = df_final[[c for c in cols_output if c in df_final.columns]]

output_path = DATA_OUTPUT / "pnad_ilo_merged.csv"
df_final.to_csv(output_path, index=False)

# ---------------------------------------------------------------------------
# Resumo final
# ---------------------------------------------------------------------------
print(f"\n{'=' * 60}")
print("BASE FINAL CONSOLIDADA")
print(f"{'=' * 60}")
print(f"Observações:       {len(df_final):,}")
print(f"Com score:         {df_final['exposure_score'].notna().sum():,}")
print(f"Cobertura:         {df_final['exposure_score'].notna().mean():.1%}")
print(f"Colunas:           {df_final.shape[1]}")
print(f"População:         {df_final['peso'].sum()/1e6:.1f} milhões")
print(f"Salvo em:          {output_path}")
print(f"Tamanho em disco:  {output_path.stat().st_size / 1e6:.1f} MB")

df_final.info()

Distribuição por gradiente:
  Not Exposed: 41.5 milhões
  Minimal Exposure: 21.1 milhões
  Gradient 1-2: 18.1 milhões
  Gradient 4 (Alta): 8.0 milhões
  Gradient 3: 5.4 milhões
  Sem classificação: 0.8 milhões

BASE FINAL CONSOLIDADA
Observações:       192,235
Com score:         190,551
Cobertura:         99.1%
Colunas:           27
População:         94.9 milhões
Salvo em:          data/output/pnad_ilo_merged.csv
Tamanho em disco:  32.9 MB
<class 'pandas.core.frame.DataFrame'>
Index: 192235 entries, 0 to 202220
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   ano                   192235 non-null  Int64   
 1   trimestre             192235 non-null  Int64   
 2   sigla_uf              192235 non-null  object  
 3   regiao                192235 non-null  object  
 4   sexo                  192235 non-null  object  
 5   sexo_texto            192235 non-null  object  
 6   idade               